In [ ]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
#       EXPERIMENT: VOLTAS PIPELINE — STEP 1: DATA PROFILING (COLAB)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

# 1) Install libraries (Colab)
!pip -q install ydata-profiling great_expectations

import pandas as pd
from ydata_profiling import ProfileReport

# 2) Upload your dataset
print("👉 Please upload your Voltas dataset CSV file (e.g., voltas_raw_scraped_like.csv).")
file_name = None
try:
    from google.colab import files
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    print(f"✅ Uploaded: {file_name}")
except Exception as e:
    print(f"⚠️ Upload step skipped or failed: {e}\n"
          f"If running locally, set file_name manually and ensure file exists.")
    # file_name = "voltas_raw_scraped_like.csv"  # <-- uncomment & set if running locally

# 3) Generate the profiling report
if file_name:
    try:
        print(f"📥 Reading '{file_name}'...")
        df_raw = pd.read_csv(file_name)
        print(f"✅ Data loaded: {df_raw.shape[0]} rows × {df_raw.shape[1]} cols")

        print("📊 Building ydata-profiling HTML report (explorative=True)...")
        profile = ProfileReport(
            df_raw,
            title="Voltas Dataset — RAW Profiling Report",
            explorative=True
        )

        raw_report = "voltas_raw_profile.html"
        profile.to_file(raw_report)
        print(f"💾 Report saved as '{raw_report}'")

        try:
            files.download(raw_report)
            print("⬇️ Report download triggered.")
        except Exception as e:
            print(f"ℹ️ Auto-download unavailable here: {e}. File saved in working directory.")

    except Exception as e:
        print(f"❌ Profiling error: {e}")
else:
    print("⛔ No file selected; skip profiling.")


👉 Please upload your Voltas dataset CSV file (e.g., voltas_raw_scraped_like.csv).


Saving voltas_raw_scraped_like.csv to voltas_raw_scraped_like.csv
✅ Uploaded: voltas_raw_scraped_like.csv
📥 Reading 'voltas_raw_scraped_like.csv'...
✅ Data loaded: 500 rows × 29 cols
📊 Building ydata-profiling HTML report (explorative=True)...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 29/29 [00:00<00:00, 46.26it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Report saved as 'voltas_raw_profile.html'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Report download triggered.


In [ ]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
#       EXPERIMENT: VOLTAS PIPELINE — STEP 2: DATA CLEANING (COLAB)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

import pandas as pd
import numpy as np

# 1) Upload (or reuse df_raw from Step 1)
if 'df_raw' not in globals():
    print("👉 Please upload your Voltas dataset CSV file (again if needed).")
    try:
        from google.colab import files
        uploaded = files.upload()
        file_name = next(iter(uploaded))
        print(f"✅ Uploaded: {file_name}")
        df_raw = pd.read_csv(file_name)
    except Exception as e:
        print(f"❌ Could not load data: {e}")
        df_raw = None

def clean_voltas_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the Voltas dataset using your original rules:
      - Drop dups on ['product_id','username','review_date'] if present
      - Parse dates
      - Fill numeric NA with median, categorical with mode
    Also keeps dtype coercions safe and prints brief stats.
    """
    if df is None or df.empty:
        raise ValueError("Empty DataFrame provided to clean_voltas_df().")

    dfc = df.copy()
    print(f"\n🧹 Cleaning start — input: {dfc.shape[0]} rows, {dfc.shape[1]} cols")

    # A) drop duplicates on key columns if present
    key_cols = [c for c in ['product_id', 'username', 'review_date'] if c in dfc.columns]
    if key_cols:
        before = len(dfc)
        dfc.drop_duplicates(subset=key_cols, inplace=True)
        print(f"• Dropped {before - len(dfc)} duplicate rows on keys {key_cols}")
    else:
        print("• No key columns found for de-dup (skipped).")

    # B) coerce dates
    for date_col in ['review_date', 'manufacturing_date']:
        if date_col in dfc.columns:
            dfc[date_col] = pd.to_datetime(dfc[date_col], errors='coerce', dayfirst=True)
            nulls = dfc[date_col].isna().sum()
            print(f"• Parsed '{date_col}' → NaT count: {nulls}")

    # C) fill numeric NA with median (column-wise)
    num_cols = dfc.select_dtypes(include='number').columns.tolist()
    if num_cols:
        for c in num_cols:
            med = dfc[c].median() if not dfc[c].dropna().empty else 0
            dfc[c] = dfc[c].fillna(med)
        print(f"• Filled numeric NA with column medians for {len(num_cols)} columns")
    else:
        print("• No numeric columns found (skipped numeric fills).")

    # D) fill categorical NA with mode
    cat_cols = dfc.select_dtypes(include='object').columns.tolist()
    if cat_cols:
        for c in cat_cols:
            if dfc[c].notna().any():
                mode_val = dfc[c].mode(dropna=True)
                mode_val = mode_val.iloc[0] if not mode_val.empty else ""
                dfc[c] = dfc[c].fillna(mode_val)
        print(f"• Filled categorical NA with mode for {len(cat_cols)} columns")
    else:
        print("• No object/categorical columns found (skipped cat fills).")

    print(f"✅ Cleaning complete — output: {dfc.shape[0]} rows, {dfc.shape[1]} cols")
    return dfc

# 2) Run cleaning + save
if 'df_raw' in globals() and df_raw is not None and not df_raw.empty:
    try:
        df_cleaned = clean_voltas_df(df_raw)
        cleaned_path = "voltas_cleaned.csv"
        df_cleaned.to_csv(cleaned_path, index=False)
        print(f"💾 Saved cleaned CSV → {cleaned_path} ({len(df_cleaned)} rows)")

        try:
            from google.colab import files
            files.download(cleaned_path)
            print("⬇️ Cleaned file download triggered.")
        except Exception as e:
            print(f"ℹ️ Auto-download unavailable here: {e}. File saved in working directory.")
    except Exception as e:
        print(f"❌ Cleaning error: {e}")
else:
    print("⛔ No df_raw available; cannot clean.")



🧹 Cleaning start — input: 500 rows, 29 cols
• Dropped 0 duplicate rows on keys ['product_id', 'username', 'review_date']
• Parsed 'review_date' → NaT count: 15
• Parsed 'manufacturing_date' → NaT count: 15
• Filled numeric NA with column medians for 10 columns
• Filled categorical NA with mode for 17 columns
✅ Cleaning complete — output: 500 rows, 29 cols
💾 Saved cleaned CSV → voltas_cleaned.csv (500 rows)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Cleaned file download triggered.


In [ ]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
#  VOLTAS PIPELINE — STEP 3: CORE + ADVANCED FEATURE ENGINEERING (COLAB)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

import pandas as pd
import numpy as np
import re

# 1) Load cleaned file (or reuse df_cleaned)
if 'df_cleaned' not in globals():
    try:
        print("📥 Loading 'voltas_cleaned.csv'...")
        df_cleaned = pd.read_csv("voltas_cleaned.csv")
    except Exception as e:
        print(f"❌ Could not load 'voltas_cleaned.csv': {e}")
        df_cleaned = None

def engineer_voltas_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Recreates your original features + adds optional advanced ones (only if columns exist):
      - review_length_words / review_length_chars
      - rating_bucket (bins: 0–2, 2–4, 4–5]
      - competitor_price_diff_percent (price_inr vs competitor_price)
      - price_density_score (price_inr / capacity_numeric extracted from capacity text)
      - warranty_quality_interaction (warranty_years * customer_rating)
      - (safe parsing) re-coerces dates if present
    """
    if df is None or df.empty:
        raise ValueError("Empty DataFrame provided to engineer_voltas_features().")

    dff = df.copy()
    print(f"\n🧪 Feature engineering start — input: {dff.shape[0]} rows")

    # A) ensure dates are datetime
    for date_col in ['review_date', 'manufacturing_date']:
        if date_col in dff.columns and not np.issubdtype(dff[date_col].dtype, np.datetime64):
            dff[date_col] = pd.to_datetime(dff[date_col], errors='coerce', dayfirst=True)

    # B) original text-based features
    if 'review_text' in dff.columns:
        dff['review_text'] = dff['review_text'].astype(str)
        dff['review_length_words'] = dff['review_text'].apply(lambda x: len(x.split()))
        dff['review_length_chars'] = dff['review_text'].apply(len)
        print("• Added review_length_words & review_length_chars")

    # C) original rating bucket
    if 'rating' in dff.columns:
        try:
            dff['rating'] = pd.to_numeric(dff['rating'], errors='coerce')
            dff['rating_bucket'] = pd.cut(
                dff['rating'],
                bins=[0, 2, 4, 5],
                labels=["Low", "Medium", "High"],
                include_lowest=True
            )
            print("• Added rating_bucket (Low/Medium/High)")
        except Exception as e:
            print(f"• rating_bucket creation skipped due to: {e}")

    # --- ADVANCED (presence-aware; will auto-skip if cols missing) ---
    # D) competitor price diff %
    if all(col in dff.columns for col in ['price_inr', 'competitor_price']):
        dff['price_inr'] = pd.to_numeric(dff['price_inr'], errors='coerce')
        dff['competitor_price'] = pd.to_numeric(dff['competitor_price'], errors='coerce')
        dff['competitor_price_diff_percent'] = (
            (dff['price_inr'] - dff['competitor_price']) / (dff['competitor_price'] + 1e-6)
        ) * 100
        print("• Added competitor_price_diff_percent")

    # E) capacity_unified (if you have scattered capacity columns)
    capacity_cols = ['capacity_tons', 'capacity_liters', 'capacity_kg', 'capacity_place_settings']
    if any(c in dff.columns for c in capacity_cols) and 'capacity_unified' not in dff.columns:
        def unify_capacity(row):
            if 'capacity_tons' in row and pd.notna(row['capacity_tons']):   return f"{row['capacity_tons']} Tons"
            if 'capacity_kg' in row and pd.notna(row['capacity_kg']):       return f"{row['capacity_kg']} Kg"
            if 'capacity_liters' in row and pd.notna(row['capacity_liters']): return f"{row['capacity_liters']} Liters"
            if 'capacity_place_settings' in row and pd.notna(row['capacity_place_settings']): return f"{row['capacity_place_settings']} Place Settings"
            return "Not Applicable"
        dff['capacity_unified'] = dff.apply(unify_capacity, axis=1)
        print("• Built capacity_unified from capacity_* columns")

    # F) price density score (price per capacity)
    if 'price_inr' in dff.columns and 'capacity_unified' in dff.columns:
        dff['capacity_numeric'] = pd.to_numeric(
            dff['capacity_unified'].astype(str).str.extract(r'(\d+\.?\d*)')[0], errors='coerce'
        )
        dff['price_density_score'] = dff['price_inr'] / (dff['capacity_numeric'] + 1e-6)
        dff['price_density_score'].replace([np.inf, -np.inf], np.nan, inplace=True)
        dff['price_density_score'] = dff['price_density_score'].fillna(0)
        dff.drop(columns=['capacity_numeric'], inplace=True)
        print("• Added price_density_score")

    # G) warranty × rating interaction
    if 'warranty_years' in dff.columns and 'customer_rating' in dff.columns:
        dff['warranty_years'] = pd.to_numeric(dff['warranty_years'], errors='coerce').fillna(0)
        dff['customer_rating'] = pd.to_numeric(dff['customer_rating'], errors='coerce').fillna(0)
        dff['warranty_quality_interaction'] = dff['warranty_years'] * dff['customer_rating']
        print("• Added warranty_quality_interaction")

    print("✅ Feature engineering complete.")
    return dff

# 2) Run & save
if 'df_cleaned' in globals() and df_cleaned is not None and not df_cleaned.empty:
    try:
        df_feat = engineer_voltas_features(df_cleaned)
        feat_path = "voltas_featured.csv"
        df_feat.to_csv(feat_path, index=False)
        print(f"💾 Saved features CSV → {feat_path} ({len(df_feat)} rows)")

        try:
            from google.colab import files
            files.download(feat_path)
            print("⬇️ Featured file download triggered.")
        except Exception as e:
            print(f"ℹ️ Auto-download unavailable here: {e}. File saved in working directory.")
    except Exception as e:
        print(f"❌ Feature engineering error: {e}")
else:
    print("⛔ No cleaned data found; cannot engineer features.")



🧪 Feature engineering start — input: 500 rows
• Built capacity_unified from capacity_* columns
• Added price_density_score
• Added warranty_quality_interaction
✅ Feature engineering complete.
💾 Saved features CSV → voltas_featured.csv (500 rows)


/tmp/ipython-input-346136456.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dff['price_density_score'].replace([np.inf, -np.inf], np.nan, inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Featured file download triggered.


In [ ]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
#       EXPERIMENT: VOLTAS — STEP 4 (ONLY): GE VALIDATION
#            (Legacy PandasDataset path — stable)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

import pandas as pd
import great_expectations as gx

# Prefer the legacy Dataset API for maximum compatibility
from great_expectations.dataset import PandasDataset

# 1) Load featured data
df_path = "voltas_featured.csv"
df = pd.read_csv(df_path)
print(f"✅ Loaded '{df_path}' → {df.shape[0]} rows × {df.shape[1]} cols")

# 2) Wrap in a GE PandasDataset
ge_df = PandasDataset(df)

# 3) Presence-aware expectations (won’t crash if a column is missing)
def col_exists(c: str) -> bool:
    return c in ge_df.columns

expected_core = [c for c in [
    'product_id','username','review_date','review_text','rating',
    'price_inr','competitor_price','warranty_years','customer_rating',
    'capacity_unified','review_length_words','review_length_chars',
    'rating_bucket','competitor_price_diff_percent','price_density_score',
    'warranty_quality_interaction'
] if col_exists(c)]

# Table-level presence (unordered, friendlier during iteration)
if expected_core:
    ge_df.expect_table_columns_to_match_set(
        column_set=expected_core, exact_match=False
    )

# Value ranges
if col_exists('rating'):
    ge_df.expect_column_values_to_be_between('rating', min_value=0, max_value=5)

if col_exists('customer_rating'):
    ge_df.expect_column_values_to_be_between('customer_rating', min_value=0, max_value=5)

if col_exists('warranty_years'):
    ge_df.expect_column_values_to_be_between('warranty_years', min_value=0)

# Not-null checks for key identifiers/dates
for c in ['product_id', 'username', 'review_date']:
    if col_exists(c):
        ge_df.expect_column_values_to_not_be_null(c)

# Categorical domain for rating bucket
if col_exists('rating_bucket'):
    ge_df.expect_column_values_to_be_in_set('rating_bucket', ['Low','Medium','High'])

# 4) Run validation
result = ge_df.validate()
print("\n📊 Validation Summary:")
print("SUCCESS ✅" if result.success else "FAILED ❌")

# Helpful diff if presence check failed
if expected_core:
    observed = set(ge_df.columns)
    expected = set(expected_core)
    missing = sorted(expected - observed)
    extra   = sorted(observed - expected)
    if missing: print(" • Missing columns:", missing)
    if extra:   print(" • Extra columns (present but not expected):", extra)

# Optional: compact counts
try:
    stats = result["statistics"]
    print(f"\nExpectations run: {stats.get('evaluated_expectations')}, "
          f"Successful: {stats.get('successful_expectations')}, "
          f"Unsuccessful: {stats.get('unsuccessful_expectations')}")
except Exception:
    pass


✅ Loaded 'voltas_featured.csv' → 500 rows × 32 cols

📊 Validation Summary:
FAILED ❌
 • Extra columns (present but not expected): ['availability', 'capacity_kg', 'capacity_liters', 'capacity_place_settings', 'capacity_tons', 'city', 'color', 'complaint_text', 'discount_offered', 'energy_rating_stars', 'feature_1', 'manufacturing_date', 'model_name', 'platform', 'product_category', 'product_name', 'resolved_status', 'return_status', 'review_sentiment', 'reviewer_location', 'sub_type', 'technology', 'warranty_duration_months']

Expectations run: 6, Successful: 5, Unsuccessful: 1


In [ ]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
#        (OPTIONAL) STEP 5: CLEANED DATA PROFILING (POST-CLEAN)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

# Create a profiling report on the CLEANED data as well (mirrors your original Sweetviz-on-cleaned step).

from ydata_profiling import ProfileReport

if 'df_cleaned' in globals() and df_cleaned is not None and not df_cleaned.empty:
    try:
        print("📊 Building ydata-profiling HTML report for CLEANED data...")
        profile_clean = ProfileReport(
            df_cleaned,
            title="Voltas Dataset — CLEANED Profiling Report",
            explorative=True
        )
        clean_report = "voltas_cleaned_profile.html"
        profile_clean.to_file(clean_report)
        print(f"💾 Report saved as '{clean_report}'")

        try:
            from google.colab import files
            files.download(clean_report)
            print("⬇️ Cleaned profiling report download triggered.")
        except Exception as e:
            print(f"ℹ️ Auto-download unavailable here: {e}. File saved in working directory.")
    except Exception as e:
        print(f"❌ Cleaned profiling error: {e}")
else:
    print("ℹ️ No df_cleaned available; skipped cleaned profiling.")


📊 Building ydata-profiling HTML report for CLEANED data...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 29/29 [00:00<00:00, 42.44it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Report saved as 'voltas_cleaned_profile.html'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Cleaned profiling report download triggered.
